# 일기 정보 추출 에이전트

사용자가 작성한 일기에서 주제, 감정 등을 추출하는 에이전트

In [9]:
# 필요한 라이브러리 import
from datetime import datetime
from typing import TypedDict, Optional, List
from langgraph.graph import StateGraph, END
import os
import json
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage

load_dotenv()
api_key = os.getenv("UPSTAGE_API_KEY")

# LLM 초기화
chat = ChatUpstage(
    model="solar-pro2-251215",
    upstage_api_key=api_key
)

In [10]:
### State 정의

class DiaryExtractionState(TypedDict):
    """일기 정보 추출 에이전트의 State"""
    diary_content: str  # 원본 일기 내용
    datetime: str  # 일기 작성 시간 "YYYY-MM-DD HH:mm:ss"
    topic: Optional[str]  # 추출된 주제 (예: "부장회의", "야식", "친구 약속")
    emotion: Optional[str]  # 추출된 감정 (예: "빡침", "슬픔", "기쁨", "걱정")

In [11]:
### 노드 함수 정의

def extract_info(state: DiaryExtractionState) -> DiaryExtractionState:
    """일기에서 주제, 감정, 키워드 등을 추출하는 노드"""
    diary_content = state["diary_content"]
    diary_datetime = state.get("datetime", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
    print(f"[extract_info] 일기 정보 추출 중...")
    print(f"  - 일기 내용: {diary_content[:100]}...")
    print(f"  - 작성 시간: {diary_datetime}")
    
    # LLM 호출을 위한 프롬프트 구성 (간단하게 주제와 감정만)
    prompt = f"""사용자가 작성한 짧은 메모에서 주제와 감정만 추출하고 JSON으로 응답하세요.

일기 내용: {diary_content}

=== 추출 항목 ===
1. topic: 주요 주제나 사건 (예: "부장회의", "야식", "친구 약속")
   - 한 단어 또는 짧은 구로 표현
   
2. emotion: 감정 (예: "빡침", "화남", "슬픔", "기쁨", "걱정", "후회")
   - 비속어나 구어체 표현도 그대로 반영 (예: "빡침")

=== 예시 ===
입력: "아 부장 ㅅㅂ 화나네 회의때깨짐"
출력: {{"topic": "부장회의", "emotion": "빡침"}}

입력: "야식 먹어서 살찌겟네 ㅠ"
출력: {{"topic": "야식", "emotion": "후회"}}

JSON 형식:
{{
  "topic": "주제",
  "emotion": "감정"
}}"""
    
    # LLM 호출
    response = chat.invoke(prompt)
    
    # JSON 파싱
    try:
        # 응답에서 JSON 추출
        response_text = response.content.strip()
        
        # JSON 부분만 추출 (```json ... ``` 또는 {...} 형식)
        if "```json" in response_text:
            json_start = response_text.find("```json") + 7
            json_end = response_text.find("```", json_start)
            response_text = response_text[json_start:json_end].strip()
        elif "```" in response_text:
            json_start = response_text.find("```") + 3
            json_end = response_text.find("```", json_start)
            response_text = response_text[json_start:json_end].strip()
        
        # 첫 번째 JSON 객체만 추출 (중괄호로 시작하는 부분)
        if "{" in response_text:
            start_idx = response_text.find("{")
            brace_count = 0
            end_idx = start_idx
            
            for i in range(start_idx, len(response_text)):
                if response_text[i] == "{":
                    brace_count += 1
                elif response_text[i] == "}":
                    brace_count -= 1
                    if brace_count == 0:
                        end_idx = i + 1
                        break
            
            response_text = response_text[start_idx:end_idx]
        
        result_json = json.loads(response_text)
        state["topic"] = result_json.get("topic", "")
        state["emotion"] = result_json.get("emotion", "")
        
        print(f"[extract_info] 추출 완료")
        print(f"  - 주제: {state['topic']}")
        print(f"  - 감정: {state['emotion']}")
        
    except Exception as e:
        print(f"[extract_info] JSON 파싱 실패: {e}")
        print(f"[extract_info] 원본 응답: {response.content[:200]}...")
        # 기본값 사용
        state["topic"] = ""
        state["emotion"] = ""
    
    return state

In [12]:
### 그래프 구성

# StateGraph 생성
workflow = StateGraph(DiaryExtractionState)

# 노드 추가
workflow.add_node("extract_info", extract_info)

# 엣지 추가 - 단순 선형 흐름
workflow.set_entry_point("extract_info")
workflow.add_edge("extract_info", END)

# 그래프 컴파일
app = workflow.compile()

print("✅ 일기 정보 추출 에이전트가 생성되었습니다!")

✅ 일기 정보 추출 에이전트가 생성되었습니다!


In [13]:
### 테스트 실행

# 테스트 케이스 1: 부장 회의 예시
test_case_1 = DiaryExtractionState(
    diary_content="아 부장 ㅅㅂ 화나네 회의때깨짐",
    datetime=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    topic=None,
    emotion=None
)

result_1 = app.invoke(test_case_1)

print("\n=== 테스트 케이스 1: 부장 회의 ===")
print(f"원본: {result_1['diary_content']}")
print(f"주제: {result_1['topic']}")
print(f"감정: {result_1['emotion']}")

[extract_info] 일기 정보 추출 중...
  - 일기 내용: 아 부장 ㅅㅂ 화나네 회의때깨짐...
  - 작성 시간: 2026-01-14 14:28:50
[extract_info] 추출 완료
  - 주제: 부장회의
  - 감정: 빡침

=== 테스트 케이스 1: 부장 회의 ===
원본: 아 부장 ㅅㅂ 화나네 회의때깨짐
주제: 부장회의
감정: 빡침


In [14]:
# 테스트 케이스 2: 야식 예시
test_case_2 = DiaryExtractionState(
    diary_content="야식 먹어서 살찌겟네 ㅠ",
    datetime=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    topic=None,
    emotion=None
)

result_2 = app.invoke(test_case_2)

print("\n=== 테스트 케이스 2: 야식 ===")
print(f"원본: {result_2['diary_content']}")
print(f"주제: {result_2['topic']}")
print(f"감정: {result_2['emotion']}")

[extract_info] 일기 정보 추출 중...
  - 일기 내용: 야식 먹어서 살찌겟네 ㅠ...
  - 작성 시간: 2026-01-14 14:28:51
[extract_info] 추출 완료
  - 주제: 야식
  - 감정: 후회

=== 테스트 케이스 2: 야식 ===
원본: 야식 먹어서 살찌겟네 ㅠ
주제: 야식
감정: 후회


In [15]:
# 테스트 케이스 3: 긴 일기 예시
test_case_3 = DiaryExtractionState(
    diary_content="오늘은 프로젝트 발표 준비를 했다. 조금 긴장되지만 잘 할 수 있을 것 같다.",
    datetime=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    topic=None,
    emotion=None
)

result_3 = app.invoke(test_case_3)

print("\n=== 테스트 케이스 3: 프로젝트 발표 ===")
print(f"원본: {result_3['diary_content']}")
print(f"주제: {result_3['topic']}")
print(f"감정: {result_3['emotion']}")

[extract_info] 일기 정보 추출 중...
  - 일기 내용: 오늘은 프로젝트 발표 준비를 했다. 조금 긴장되지만 잘 할 수 있을 것 같다....
  - 작성 시간: 2026-01-14 14:28:52
[extract_info] 추출 완료
  - 주제: 프로젝트 발표 준비
  - 감정: 긴장

=== 테스트 케이스 3: 프로젝트 발표 ===
원본: 오늘은 프로젝트 발표 준비를 했다. 조금 긴장되지만 잘 할 수 있을 것 같다.
주제: 프로젝트 발표 준비
감정: 긴장


In [16]:
### 추출 함수 (간편 사용을 위한 래퍼 함수)

def extract_diary_info(diary_content: str, diary_datetime: str = None) -> dict:
    """
    일기 내용에서 주제와 감정을 추출하는 간편 함수
    
    Args:
        diary_content: 일기 원본 내용
        diary_datetime: 일기 작성 시간 (선택, 기본값: 현재 시간)
    
    Returns:
        추출된 정보 딕셔너리 (topic, emotion 포함)
    """
    if diary_datetime is None:
        diary_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    initial_state = DiaryExtractionState(
        diary_content=diary_content,
        datetime=diary_datetime,
        topic=None,
        emotion=None
    )
    
    result = app.invoke(initial_state)
    
    return {
        "topic": result.get("topic", ""),
        "emotion": result.get("emotion", ""),
        "datetime": result.get("datetime", diary_datetime)
    }

# 사용 예시
print("\n=== 추출 함수 사용 예시 ===")
extracted = extract_diary_info("아 부장 ㅅㅂ 화나네 회의때깨짐")
print(f"추출 결과: {extracted}")


=== 추출 함수 사용 예시 ===
[extract_info] 일기 정보 추출 중...
  - 일기 내용: 아 부장 ㅅㅂ 화나네 회의때깨짐...
  - 작성 시간: 2026-01-14 14:28:52
[extract_info] 추출 완료
  - 주제: 부장회의
  - 감정: 화남
추출 결과: {'topic': '부장회의', 'emotion': '화남', 'datetime': '2026-01-14 14:28:52'}
